In [ ]:
!nvidia-smi

Mon Feb 21 07:45:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/semeval_task_5.zip
! mv /content/semeval_task_5/InteractiveAttention.py /content
! mv /content/semeval_task_5/img_features.pkl /content
! mv /content/semeval_task_5/TRAINING /content

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: semeval_task_5/TRAINING/4665.jpg  
  inflating: semeval_task_5/TRAINING/4666.jpg  
  inflating: semeval_task_5/TRAINING/4667.jpg  
  inflating: semeval_task_5/TRAINING/4669.jpg  
  inflating: semeval_task_5/TRAINING/467.jpg  
  inflating: semeval_task_5/TRAINING/4670.jpg  
  inflating: semeval_task_5/TRAINING/4671.jpg  
  inflating: semeval_task_5/TRAINING/4673.jpg  
  inflating: semeval_task_5/TRAINING/4674.jpg  
  inflating: semeval_task_5/TRAINING/4675.jpg  
  inflating: semeval_task_5/TRAINING/4678.jpg  
  inflating: semeval_task_5/TRAINING/4679.jpg  
  inflating: semeval_task_5/TRAINING/468.jpg  
  inflating: semeval_task_5/TRAINING/4680.jpg  
  inflating: semeval_task_5/TRAINING/4681.jpg  
  inflating: semeval_task_5/TRAINING/4682.jpg  
  inflating: semeval_task_5/TRAINING/4683.jpg  
  inflating: semeval_task_5/TRAINING/4685.jpg  
  inflating: semeval_task_5/TRAINING/4686.jpg  
  inflating: semeval_task_5/TRAINING/4687.jpg  
  inflating: se

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 4.3 MB/s 
     |████████████████████████████████| 67 kB 7.9 MB/s 
     |████████████████████████████████| 596 kB 92.1 MB/s 
     |████████████████████████████████| 895 kB 61.0 MB/s 
     |████████████████████████████████| 6.8 MB 51.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 4.1 MB/s 


In [ ]:
import logging
import os
import sys
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import LxmertTokenizerFast, LxmertConfig
from transformers.models.lxmert.modeling_tf_lxmert import TFLxmertMainLayer
from InteractiveAttention import InteractiveAttention
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split

batch_size = 16
epochs = 20
num_labels = 5

program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info(r"running %s" % ''.join(sys.argv))
data_path = os.path.join('TRAINING/training.csv')
train_df = pd.read_csv(data_path, sep=',')
train_texts, train_labels = [], []
classification = ['misogynous', 'shaming', 'stereotype', 'objectification', 'violence']  # labels of the dataset
img_ROI_train = joblib.load('img_features.pkl')

visual_feats = []
visual_pos = []

for i, identity in enumerate(train_df['file_name']):
    # load image
    visual_feats.append(img_ROI_train[str(identity)]['features'])
    visual_pos.append(img_ROI_train[str(identity)]['boxes'])
    # load text
    train_texts.append(train_df['Text Transcription'][i])
    # load labels
    emotion_class = np.zeros(5)
    for j, emotion in enumerate(classification):
        if train_df[emotion][i] == 1:
            emotion_class[j] = 1
    train_labels.append(emotion_class.tolist())

visual_feats_train, visual_feats_val, visual_pos_train, visual_pos_val, train_texts, val_texts, train_labels, val_labels = \
    train_test_split(visual_feats, visual_pos, train_texts, train_labels, random_state=10, test_size=0.1)
tokenizer = LxmertTokenizerFast.from_pretrained('unc-nlp/lxmert-vqa-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding='max_length', max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding='max_length', max_length=128)

print(np.array(train_encodings['input_ids']).shape)
print(np.array(val_encodings['input_ids']).shape)

train_dataset = tf.data.Dataset.from_tensor_slices((
    {'visual_feats': visual_feats_train,
     'visual_pos': visual_pos_train,
     'input_ids': train_encodings['input_ids'],
     'token_type_ids': train_encodings['token_type_ids'],
     'attention_mask': train_encodings['attention_mask']},
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    {'visual_feats': visual_feats_val,
     'visual_pos': visual_pos_val,
     'input_ids': val_encodings['input_ids'],
     'token_type_ids': val_encodings['token_type_ids'],
     'attention_mask': val_encodings['attention_mask']},
    val_labels
))

config = LxmertConfig.from_pretrained('unc-nlp/lxmert-vqa-uncased', max_position_embeddings=128)

visual_feats = tf.keras.Input(shape=(36, config.visual_feat_dim,), dtype='float32')
visual_pos = tf.keras.Input(shape=(36, config.visual_pos_dim,), dtype='float32')
input_ids = tf.keras.Input(shape=(config.max_position_embeddings,), dtype='int32')
token_type_ids = tf.keras.Input(shape=(config.max_position_embeddings,), dtype='int32')
attention_mask = tf.keras.Input(shape=(config.max_position_embeddings,), dtype='int32')

inputs = {'visual_feats': visual_feats,
          'visual_pos': visual_pos,
          'input_ids': input_ids,
          'token_type_ids': token_type_ids,
          'attention_mask': attention_mask}

lxmert_layer = TFLxmertMainLayer(config=config)
# lxmert_output = lxmert_layer(inputs)

lxmert_output_lang = lxmert_layer(inputs)[2]  # 768
lxmert_output_visual = lxmert_layer(inputs)[1]  # 36 * 768
lxmert_output_visual = tf.keras.layers.Flatten()(lxmert_output_visual)
# lxmert_output_visual = tf.keras.layers.MaxPool1D(
#     pool_size=(36), strides=None, padding='valid')(lxmert_output_visual)
# lxmert_output_visual = tf.squeeze(
#     lxmert_output_visual, axis=1, name=None
# )
lxmert_output_visual = tf.keras.layers.Dense(768, activation='relu')(
    lxmert_output_visual)
lxmert_output_visual = tf.keras.layers.Dropout(config.hidden_dropout_prob)(lxmert_output_visual)

print("attention!\n")

print(lxmert_output_lang.shape)
print(lxmert_output_visual.shape)
# concate = tf.keras.layers.concatenate([lxmert_output_lang, lxmert_output_visual], axis=-1)
# dropout_output = tf.keras.layers.Dropout(config.hidden_dropout_prob)(concate)
lxmert_output_visual = tf.keras.layers.LayerNormalization(epsilon=1e-6)(lxmert_output_visual)
lxmert_output_lang = tf.keras.layers.LayerNormalization(epsilon=1e-6)(lxmert_output_lang)
merge = InteractiveAttention(input_dim=lxmert_output_visual.shape[-1], input_len=lxmert_output_lang.shape[-1])(
    lxmert_output_visual, lxmert_output_lang)
dropout_output = tf.keras.layers.Dropout(config.hidden_dropout_prob)(merge)

output = tf.keras.layers.Dense(num_labels, activation='sigmoid',
                               kernel_initializer=tf.keras.initializers.TruncatedNormal(config.initializer_range))(
    dropout_output)
model = tf.keras.Model(inputs=inputs, outputs=output)

print(model.summary())
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, amsgrad=True)
loss = tfa.losses.SigmoidFocalCrossEntropy(
    from_logits=False,
    alpha=0.35,
    gamma=3.0,
    name='sigmoid_focal_crossentropy')
f1score = tfa.metrics.F1Score(num_classes=5, threshold=0.5, average='weighted')

model.compile(optimizer=optimizer, loss=loss, metrics=['mae', 'acc', f1score])

# /content/drive/MyDrive
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Checkpoint_50epochs/'+'weights.{epoch:02d}-{val_acc:.2f}.hdf5',
#                                                                monitor='val_acc', verbose=1,
#                                                                save_best_only=False,
#                                                                save_weights_only=False, mode='auto')
# history = model.fit(train_dataset.shuffle(1000).batch(batch_size), validation_data=val_dataset.batch(batch_size),
#                     epochs=epochs, batch_size=batch_size, callbacks=[model_checkpoint_callback])
history = model.fit(train_dataset.shuffle(1000).batch(batch_size), validation_data=val_dataset.batch(batch_size),
                    epochs=epochs)


2022-02-21 07:46:35,273: INFO: running /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py-f/root/.local/share/jupyter/runtime/kernel-4a1e3262-0aab-4cdf-b260-133c7c73196d.json


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/880 [00:00<?, ?B/s]

(9000, 128)
(1000, 128)
attention!

(None, 768)
(None, 768)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 36, 2048)]   0           []                               
                                  

In [ ]:
Epoch 1/20
563/563 [==============================] - 548s 839ms/step - loss: 0.3205 - mae: 0.3941 - acc: 0.5041 - f1_score: 0.2836 - val_loss: 0.1736 - val_mae: 0.3492 - val_acc: 0.8580 - val_f1_score: 0.1917
Epoch 2/20
563/563 [==============================] - 459s 815ms/step - loss: 0.1755 - mae: 0.3873 - acc: 0.6172 - f1_score: 0.3611 - val_loss: 0.1689 - val_mae: 0.3610 - val_acc: 0.3560 - val_f1_score: 0.1295
Epoch 3/20
563/563 [==============================] - 459s 815ms/step - loss: 0.1600 - mae: 0.3881 - acc: 0.6814 - f1_score: 0.3708 - val_loss: 0.1338 - val_mae: 0.3683 - val_acc: 1.0000 - val_f1_score: 0.3418
Epoch 4/20
563/563 [==============================] - 459s 815ms/step - loss: 0.1515 - mae: 0.3853 - acc: 0.7167 - f1_score: 0.3769 - val_loss: 0.1336 - val_mae: 0.3699 - val_acc: 0.9390 - val_f1_score: 0.3293
Epoch 5/20
563/563 [==============================] - 459s 816ms/step - loss: 0.1380 - mae: 0.3695 - acc: 0.7301 - f1_score: 0.4323 - val_loss: 0.1265 - val_mae: 0.3325 - val_acc: 1.0000 - val_f1_score: 0.3474
Epoch 6/20
563/563 [==============================] - 459s 815ms/step - loss: 0.1154 - mae: 0.3268 - acc: 0.7664 - f1_score: 0.5268 - val_loss: 0.1311 - val_mae: 0.3400 - val_acc: 1.0000 - val_f1_score: 0.4643
Epoch 7/20
563/563 [==============================] - 459s 815ms/step - loss: 0.0987 - mae: 0.2917 - acc: 0.8241 - f1_score: 0.5848 - val_loss: 0.1328 - val_mae: 0.3146 - val_acc: 0.7270 - val_f1_score: 0.4570
Epoch 8/20
563/563 [==============================] - 459s 815ms/step - loss: 0.0902 - mae: 0.2654 - acc: 0.7519 - f1_score: 0.6204 - val_loss: 0.1768 - val_mae: 0.2860 - val_acc: 0.8910 - val_f1_score: 0.4836
Epoch 9/20
563/563 [==============================] - 459s 815ms/step - loss: 0.0817 - mae: 0.2444 - acc: 0.7833 - f1_score: 0.6650 - val_loss: 0.2248 - val_mae: 0.2624 - val_acc: 0.9930 - val_f1_score: 0.5134
Epoch 10/20
563/563 [==============================] - 459s 815ms/step - loss: 0.0716 - mae: 0.2206 - acc: 0.8141 - f1_score: 0.7139 - val_loss: 0.2716 - val_mae: 0.2424 - val_acc: 0.9620 - val_f1_score: 0.5512
Epoch 11/20
563/563 [==============================] - 459s 815ms/step - loss: 0.0608 - mae: 0.2021 - acc: 0.8150 - f1_score: 0.7782 - val_loss: 0.2051 - val_mae: 0.2717 - val_acc: 0.9620 - val_f1_score: 0.5826
Epoch 12/20
563/563 [==============================] - 459s 815ms/step - loss: 0.0513 - mae: 0.1831 - acc: 0.7636 - f1_score: 0.8308 - val_loss: 0.3376 - val_mae: 0.2429 - val_acc: 0.9110 - val_f1_score: 0.5935
Epoch 13/20
563/563 [==============================] - 459s 816ms/step - loss: 0.0402 - mae: 0.1607 - acc: 0.7914 - f1_score: 0.8782 - val_loss: 0.4165 - val_mae: 0.2183 - val_acc: 0.8900 - val_f1_score: 0.5944
Epoch 14/20
563/563 [==============================] - 459s 816ms/step - loss: 0.0314 - mae: 0.1400 - acc: 0.7440 - f1_score: 0.9122 - val_loss: 0.3552 - val_mae: 0.2531 - val_acc: 0.9980 - val_f1_score: 0.6023
Epoch 15/20
563/563 [==============================] - 459s 815ms/step - loss: 0.0270 - mae: 0.1264 - acc: 0.7866 - f1_score: 0.9287 - val_loss: 0.3512 - val_mae: 0.2378 - val_acc: 0.9810 - val_f1_score: 0.5838
Epoch 16/20
563/563 [==============================] - 459s 815ms/step - loss: 0.0200 - mae: 0.1116 - acc: 0.7784 - f1_score: 0.9506 - val_loss: 0.5184 - val_mae: 0.2176 - val_acc: 0.9820 - val_f1_score: 0.5911
Epoch 17/20
563/563 [==============================] - 459s 815ms/step - loss: 0.0156 - mae: 0.0911 - acc: 0.6800 - f1_score: 0.9617 - val_loss: 0.5172 - val_mae: 0.2174 - val_acc: 0.8280 - val_f1_score: 0.5822
Epoch 18/20
563/563 [==============================] - 459s 816ms/step - loss: 0.0114 - mae: 0.0772 - acc: 0.6487 - f1_score: 0.9728 - val_loss: 0.4811 - val_mae: 0.2275 - val_acc: 0.7300 - val_f1_score: 0.5842
Epoch 19/20
563/563 [==============================] - 459s 815ms/step - loss: 0.0116 - mae: 0.0749 - acc: 0.5713 - f1_score: 0.9730 - val_loss: 0.5561 - val_mae: 0.2083 - val_acc: 0.8570 - val_f1_score: 0.5790
Epoch 20/20
563/563 [==============================] - 459s 816ms/step - loss: 0.0101 - mae: 0.0706 - acc: 0.5120 - f1_score: 0.9781 - val_loss: 0.6653 - val_mae: 0.2098 - val_acc: 0.8040 - val_f1_score: 0.5750